In [105]:
from pathlib import Path
import csv
import pandas as pd
from datetime import datetime

In [106]:
# loading the data

notebook_dir = Path.cwd()
# the data is located in data/raw folder
data_dir = notebook_dir.parent.absolute() / 'data_raw' / 'instagram_data'
csv_dir_grouped = notebook_dir.parent.absolute() / 'clean_csvs' / 'messages_grouped_by_user'
csv_dir_separate = notebook_dir.parent.absolute() / 'clean_csvs' / 'messages_separate'

#convert txt of instagram texts to strings
path_raw_1 = data_dir / 'aarush_instagram_message_1.txt'
path_raw_2 = data_dir / 'aarush_instagram_message_2.txt'
path_raw = data_dir / 'aarush_full_instagram_messages.txt'
path_csv_grouped = csv_dir_grouped / 'aarush-instagram-grouped.csv'
path_csv_separate = csv_dir_separate / 'aarush-instagram-separate.csv'

In [107]:
# merging the two text files

def cleanTxtFile(txt):
    firstInd = txt.find('12\n12')
    txt = txt[firstInd + 5:].strip()
    return(txt)

data = data2 = ''
# Reading data from file1
with open(path_raw_1) as fp:
    data = fp.read()

data = cleanTxtFile(data)

with open(path_raw_2) as fp:
    data2 = fp.read()

data2 = cleanTxtFile(data2)

data += "\n"
data += data2
special_key = 'Nnọọ' #when would we ever say hello in Igbo in our chat? it's more likely than you think
data = data.replace('\naarush\n', '\n' + special_key + 'aarush\n')
data = data.replace('\nSamyukta\n', '\n' + special_key + 'Samyukta\n')
data = data.replace('\n12\n12','')
with open (path_raw, 'w') as fp:
    fp.write(data)


In [108]:
print(data.count(special_key))

16623


In [109]:
filename = "output.txt"

# Open a file in write mode ('w') and write the string to it
with open(filename, 'w') as file:
    file.write(data)


print(data.count(special_key))
print(f"String has been written to {filename}")

16623
String has been written to output.txt


In [110]:
def getReaction(line):
    asteriskIndex = line.find('* ')
    line = line[asteriskIndex + 2:]
    line = line.replace('Samyukta', '')
    line = line.replace('aarush', '')
    return line

In [111]:
def getWordCount(messageLine):
    messageLine = messageLine.strip()
    return(len(messageLine.split(' ')))

In [112]:
# figuring out how to read the instagram data
#columns for csv:
#'date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'type of reply', 'platform'
#type of reply options: story reply, reel, normal message

users = {'Samyukta', 'aarush'}
months = {'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}
user, date, text, word_count, num_sep_texts, reactions, reply_type = '', '', '', '', '', '', ''

message_dict = []

for message_block in reversed(data.split(special_key)): #split data by user
    if message_block.find('Liked a message') != -1:
        continue
    #if message_block contains "sent an attachment", it's a reel
    if message_block.find('sent an attachment.') != -1:
        reply_type = 'reel'

    #elif message_block contains https://, it's probably a story reply
    elif message_block.find('https://') != -1:
        reply_type = 'story reply'

    #else it's a normal text
    else:
        reply_type = 'normal'


    for line in message_block.splitlines():
        line = line.strip()

        #check if it's a user line
        if line in users:
            user = line

        #check if it's a date line
        elif line[:3] in months:
            date = line
        
        #check if it's a reaction line
        elif (line.find('* ') != -1) and (line.find('Samyukta') != -1 or line.find('aarush') != -1):
            reactions = getReaction(line)
        
        elif reply_type != 'reel' and (line.find('https://') == -1):
            text = line
            word_count = getWordCount(line)
    if text == '':
        reply_type = 'attachment'
    
    res = {
        'date' : date, 
        'user' : user, 
        'text' : text, 
        'word count' : word_count, 
        'number of separated texts' : num_sep_texts, 
        'reactions' : reactions, 
        'type of reply' : reply_type, 
        'platform' : 'Instagram'
        }
    user, date, text, word_count, num_sep_texts, reactions, reply_type = '', '', '', '', '', '', ''
    message_dict.append(res)
        
print(len(message_dict))
#split by user, add a special key to the start of the user so that i can do split(special_key)

#possible replies

#story replies

#sending reels

#reactions

16240


In [113]:
#save dictionary as csv
aarushIgCSV = open(path_csv_grouped, 'w')
writer = csv.writer(aarushIgCSV)
writer.writerow(['date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'type of reply', 'platform'])
for dictionary in message_dict:
    writer.writerow(dictionary.values())
aarushIgCSV.close()
aarushIgCSV = open(path_csv_grouped, 'r')
print("The content of the csv file is:")
print(aarushIgCSV.read())
aarushIgCSV.close()

The content of the csv file is:
date,user,text,word count,number of separated texts,reactions,type of reply,platform

"Oct 18, 2019, 9:23 PM",Samyukta,make both an option,4,,,normal,Instagram

"Oct 18, 2019, 9:23 PM",aarush,I mean if I were to prioritize one,8,,,normal,Instagram

"Oct 18, 2019, 9:23 PM",aarush,that's what I was asking,5,,,normal,Instagram

"Oct 18, 2019, 9:23 PM",Samyukta,Oh your comics are hella funny,6,,,normal,Instagram

"Oct 18, 2019, 9:24 PM",aarush,thx :)),2,,❤️,normal,Instagram

"Oct 19, 2019, 4:30 PM",aarush,ofc. I would reccomend a fursuit if you really wanna impress,11,,❤️,story reply,Instagram

"Oct 19, 2019, 8:49 PM",Samyukta,damn 🔥🔥,2,,,normal,Instagram

"Oct 20, 2019, 7:38 PM",aarush,is your fursuit coming together,5,,❤️,story reply,Instagram

"Oct 20, 2019, 7:43 PM",Samyukta,Yes tanvi knows nothing 😳,5,,❤️,normal,Instagram

"Oct 20, 2019, 7:43 PM",aarush,perfect,1,,❤️,normal,Instagram

"Oct 25, 2019, 12:59 PM",aarush,hey are you taking the bus to warwick

In [114]:
ig_df = pd.read_csv(path_csv_grouped)

print(ig_df.duplicated().sum())
#drop duplicates
#ig_df.drop_duplicates(inplace = True)

#make sure there are no duplicates left
print(ig_df.duplicated().sum())

489
489


In [115]:
df = ig_df
duplicated_rows = df[df.duplicated(keep=False)]  # keep=False keeps all duplicates

# Step 3: Print some of the duplicated rows
print(duplicated_rows.head())  # Display the first few duplicated rows

                      date    user text  word count  \
123  Nov 15, 2019, 6:56 PM  aarush  NaN         1.0   
124  Nov 15, 2019, 6:56 PM  aarush  NaN         1.0   
125  Nov 15, 2019, 6:56 PM  aarush  NaN         1.0   
126  Nov 15, 2019, 6:56 PM  aarush  NaN         1.0   
127  Nov 15, 2019, 6:56 PM  aarush  NaN         1.0   

     number of separated texts reactions type of reply   platform  
123                        NaN       NaN    attachment  Instagram  
124                        NaN       NaN    attachment  Instagram  
125                        NaN       NaN    attachment  Instagram  
126                        NaN       NaN    attachment  Instagram  
127                        NaN       NaN    attachment  Instagram  


In [116]:
#convert date from Nov 28, 2019, 10:12 PM to 2019-11-28 22:12:00 format
# Function to convert date format
def convert_date_format(date_str):
    try:
        # Convert the date string to a datetime object
        dt = datetime.strptime(date_str, '%b %d, %Y, %I:%M %p')
        # Convert datetime object to the desired format
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        # If the date format is incorrect or not parseable, return the original string
        return date_str

# Apply the conversion function to every column
ig_df['date'] = ig_df['date'].apply(lambda x: convert_date_format(x) if isinstance(x, str) else x)

print(ig_df.head())

                  date      user                                text  \
0  2019-10-18 21:23:00  Samyukta                 make both an option   
1  2019-10-18 21:23:00    aarush  I mean if I were to prioritize one   
2  2019-10-18 21:23:00    aarush            that's what I was asking   
3  2019-10-18 21:23:00  Samyukta      Oh your comics are hella funny   
4  2019-10-18 21:24:00    aarush                             thx :))   

   word count  number of separated texts reactions type of reply   platform  
0         4.0                        NaN       NaN        normal  Instagram  
1         8.0                        NaN       NaN        normal  Instagram  
2         5.0                        NaN       NaN        normal  Instagram  
3         6.0                        NaN       NaN        normal  Instagram  
4         2.0                        NaN        ❤️        normal  Instagram  


In [117]:
#date,user,text,word count,reactions,platform,group,number of separated texts
print(ig_df.columns)
df = ig_df[['date', 'user', 'text', 'word count', 'reactions', 'type of reply', 'platform']]

df.to_csv(path_csv_separate)

Index(['date', 'user', 'text', 'word count', 'number of separated texts',
       'reactions', 'type of reply', 'platform'],
      dtype='object')


In [118]:
# get number of separated texts

#'date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'game pigeon', 'platform'
#group all the texts from one user that are next to each other in the csv. 
#the text column should have all the texts from that user combined into one string and separated by \n
#the number of separated texts column should have the number of rows that were merged

# Function to combine consecutive messages from the same user
def combine_consecutive_messages(df):
    # Create a column to mark groups of consecutive rows with the same user
    df['group'] = (df['user'] != df['user'].shift()).cumsum()
    
    # Group by 'user' and 'group', and aggregate the texts and count of messages
    result_df = df.groupby(['user', 'group']).agg({
        'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
        'user': 'size'  # Count number of rows merged
    }).rename(columns={'user': 'number of separated texts'})
    
    # Reset index to convert 'user' and 'group' to columns
    result_df = result_df.reset_index()
    
    # Drop the 'group' column as it is no longer needed
    result_df = result_df.drop(columns=['group'])
    
    return result_df
ig_df['text'] = ig_df['text'].fillna('')
ig_df['group'] = (ig_df['user'] != ig_df['user'].shift()).cumsum()
ig_df['number of separated texts'] = ig_df['group']
result_df = ig_df.groupby(['user', 'group']).agg({
    'date' : 'first',
    'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
    'word count' : 'sum',
    'user': 'size',  # Count number of rows merged,
    'reactions' : 'first',
    'type of reply' : 'first',
    'platform' : 'first'
}).rename(columns={'user': 'number of separated texts'})

#['date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'type of reply', 'platform']

# Reset index to convert group columns back to regular columns
result_df = result_df.reset_index()
result_df = result_df.sort_values(by='date')
result_df = result_df.drop('group', axis = 1)

In [119]:
#swap the order of columns so that date comes first
new_column_order = ['date', 'user'] + [col for col in result_df.columns if col not in ['date', 'user']]

# Reorder DataFrame columns
result_df = result_df[new_column_order]

In [120]:
result_df.to_csv(path_csv_grouped)